In [10]:
import nltk # Natural Language Toolkit
nltk.download('stopwords')
# nltk.download('punkt') # Download the punkt tokenizer
# nltk.download('wordnet') # Download the wordnet lemmatizer
# from nltk import sent_tokenize, word_tokenize   # Tokenizer
# from nltk.stem.snowball import SnowballStemmer # Stemmer
# from nltk.stem.wordnet import WordNetLemmatizer # Lemmatizer
from nltk.corpus import stopwords # Stopwords
from textblob import TextBlob # TextBlob
import pandas as pd
import numpy as np 
import re   # Regular expressions
import spacy
# import en_core_web_sm
# nlp = en_core_web_sm.load()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cindydunn/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## i have
- so i figured out how to webscrape and clean the tweets now we need to pick a list of top new sources and a given time period of tweets to pull and then we can start to do some analysis.
- figured out how to pull virality metrics from the tweets. things link likes shares reports etc. and followr count at time of tweet should help me normalize the data

## what i need to do
- i will also need to figure out how to normalize these metrics based on the number of followers the account has.

In [3]:
#list of news accounts to scrape
news_accounts = ['BBCBreaking', 'cnnbrk', 'CNBCnow', 'FoxNews', 'CBSNews', 'SkyNews', 'gmanewsbreaking']

# #get todays date
from datetime import date
today = date.today()

# #get the date 7 days ago
# from datetime import datetime, timedelta
# seven_days_ago = datetime.now() - timedelta(days=7)

# today = today.strftime("%Y-%m-%d")
# seven_days_ago = seven_days_ago.strftime("%Y-%m-%d")
# print(today)
# print(seven_days_ago)

from datetime import date
# make date variable of date January 10, 2020
start_date = date(2020, 1, 10)



#generate a list of stings of news accounts and todays date and 7 days ago
news_accounts_and_dates = []
for account in news_accounts:
    news_accounts_and_dates.append('(from:' + account + ") until:" + today + " since:" + start_date)

print(news_accounts_and_dates[0])

#scrape the tweets for each account given the list of strings
import snscrape.modules.twitter as sntwitter
import pandas as pd
tweets = []
limit = 5000

for i in range(len(news_accounts_and_dates)):
    for tweet in sntwitter.TwitterSearchScraper(news_accounts_and_dates[i]).get_items():
    
        # print(vars(tweet))
        # break
        if len(tweets) == limit:
            break
        else:
            tweets.append([tweet.date, tweet.user.username, tweet.likeCount, tweet.user.followersCount, tweet.retweetCount, tweet.content])


2022-10-19
2022-10-12
(from:BBCBreaking) until:2022-10-19 since:2022-10-12


In [4]:
# #untoched code that works for scraping one twitter account
# import snscrape.modules.twitter as sntwitter
# import pandas as pd



# query = "(from:elonmusk) until:2020-01-01 since:2010-01-01"
# tweets = []
# limit = 5000


# for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
#     # print(vars(tweet))
#     # break
#     if len(tweets) == limit:
#         break
#     else:
#         tweets.append([tweet.date, tweet.user.username, tweet.likeCount, tweet.user.followersCount, tweet.retweetCount, tweet.content])
        

In [5]:
df = pd.DataFrame(tweets)
df.columns = ['date', 'username', 'likes', 'followers', 'retweets', 'content']
print(df)

                          date         username  likes  followers  retweets  \
0    2022-10-18 13:02:11+00:00      BBCBreaking   1221   51349872       514   
1    2022-10-18 09:55:10+00:00      BBCBreaking   1164   51349872       453   
2    2022-10-17 21:07:30+00:00      BBCBreaking    470   51349872       178   
3    2022-10-17 21:02:11+00:00      BBCBreaking   1449   51349872       447   
4    2022-10-17 16:32:51+00:00      BBCBreaking   1095   51349872       311   
...                        ...              ...    ...        ...       ...   
3556 2022-10-17 04:32:18+00:00  gmanewsbreaking     73    1721474         9   
3557 2022-10-17 02:48:47+00:00  gmanewsbreaking     24    1721474         6   
3558 2022-10-14 04:25:28+00:00  gmanewsbreaking      0    1721474         0   
3559 2022-10-14 00:05:56+00:00  gmanewsbreaking      6    1721474         4   
3560 2022-10-14 00:04:51+00:00  gmanewsbreaking     55    1721474        17   

                                                con

In [27]:
# for each word in the column 'Tweet' create a new column with the words in a list
df['list_of_words_in_tweet'] = df['content'].str.split()
# print(df['list_of_words_in_tweet'][3])

# remove any links from the tweets
df['list_of_words_in_tweet'] = df['list_of_words_in_tweet'].apply(lambda x: [item for item in x if 'http' not in item])

# remove any punctuation from the words in the list
df['list_of_words_in_tweet'] = df['list_of_words_in_tweet'].apply(lambda x: [re.sub(r'[^A-Za-z0-9]+','',word) for word in x])
# print(df['list_of_words_in_tweet'][3])

# remove any empty strings from the list
df['list_of_words_in_tweet'] = df['list_of_words_in_tweet'].apply(lambda x: [word for word in x if word != ''])
# print(df['list_of_words_in_tweet'][3])

# # import these modules
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize
# ps = PorterStemmer()

# # stem the words in the list
# df['list_of_words_in_tweet'] = df['list_of_words_in_tweet'].apply(lambda x: [ps.stem(word) for word in x])

#remove stop words
stop_words = set(stopwords.words('english'))
df['list_of_words_in_tweet'] = df['list_of_words_in_tweet'].apply(lambda x: [word for word in x if word not in stop_words])

# create a new column called cleaned content that is the list of words joined together into a string
# df['cleaned_content'] = df['list_of_words_in_tweet'].apply(lambda x: ' '.join(x))
# print(df['cleaned_content'][3])
# print(df['list_of_words_in_tweet'][3])

lead conserv next gener elect uk pm liz truss say stick around want say sorri mistak made httpstcotf3eva4izm httpstcovqxhf61ssu
['lead', 'conserv', 'next', 'gener', 'elect', 'uk', 'pm', 'liz', 'truss', 'say', 'stick', 'around', 'want', 'say', 'sorri', 'mistak', 'made', 'httpstcotf3eva4izm', 'httpstcovqxhf61ssu']


In [28]:
# here we are going to find subjectivity and polarity of each tweet and create a new column for each
# subjectivity is a float which lies in the range of [0,1] and means how subjective or opinionated the tweet is
# polarity is a float which lies in the range of [-1,1] and means how positive or negative the tweet is
# we use textblob for this
from textblob import TextBlob

df['subjectivity'] = df['content'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
df['polarity'] = df['content'].apply(lambda x: TextBlob(x).sentiment.polarity)

## create and connect to a SQLlite database

In [7]:
#connect to SQLlite database
import sqlite3
conn = sqlite3.connect('tweets.db') #create a database called test.db
c = conn.cursor() #create a cursor object

# check if table exists
listOfTables = c.execute(
  """SELECT name FROM sqlite_master WHERE type='table'
  AND name='twitter_accounts'; """).fetchall()
 
if listOfTables == []:
    print('Table not found!')
    #create a new table in the database called twitter_accounts with a primary key called user_id that is an integer and a column called username that is a string
    c.execute('''CREATE TABLE twitter_accounts
                (user_id integer primary key, username text)''')
else:
    print('Table found!')

#make a dataframe with each unquie twitter account
df_accounts = df['username'].unique()
df_accounts = pd.DataFrame(df_accounts, columns=['username'])
#insert the username and user_id into the table from the dataframe
for index, row in df_accounts.iterrows():
    c.execute("INSERT INTO twitter_accounts VALUES (?, ?)", (index, row['username']))
#print out the table
c.execute("SELECT * FROM twitter_accounts")
print("users in the database:")
print(c.fetchall())

#make a new table called tweets that has a primary key called tweet_id that is an integer and a column called content that is a string and a column foreign key called user_id that is an integer and a column called date that is a string and a column called likes that is an integer and a column called followers that is an integer and a column called retweets that is an integer
c.execute('''CREATE TABLE tweets
                (tweet_id integer primary key, content text, user_id integer, date text, likes integer, followers integer, retweets integer, FOREIGN KEY (user_id) REFERENCES twitter_accounts(user_id))''') 

#change the structure of df
# make a new column called user_id that is the index of the username in the df_accounts dataframe
df['user_id'] = df['username'].apply(lambda x: df_accounts[df_accounts['username'] == x].index[0])
#convert the date column to a string
df['date'] = df['date'].apply(lambda x: str(x))

#insert the content, user_id, date, likes, followers, and retweets into the table from the dataframe
for index, row in df.iterrows():
    c.execute("INSERT INTO tweets VALUES (?, ? ?, ?, ?, ?, ?, ?)", (index, row['content'] row['list_of_words_in_tweet'], row['user_id'], row['date'], row['likes'], row['followers'], row['retweets']))

#print out the table
c.execute("SELECT * FROM tweets")
print("tweets in the database:")
print(c.fetchall())

c.close()


Table not found!
users in the database:
[(0, 'BBCBreaking'), (1, 'cnnbrk'), (2, 'CNBCnow'), (3, 'FoxNews'), (4, 'CBSNews'), (5, 'SkyNews'), (6, 'gmanewsbreaking')]
tweets in the database:
[(0, 'robbi_coltran_obituari_actor_who_could_be_funni_or_serious_but_alway_compel_httpstco2bsp4m0crm', 0, '2022-10-14 17:27:32+00:00', 2160, 51329496, 329), (1, 'actor_robbi_coltran_who_play_hagrid_in_the_harri_potter_film_has_die_at_72_httpstcoayotjsixqu', 0, '2022-10-14 16:53:47+00:00', 35135, 51329496, 10746), (2, 'the_way_we_are_deliv_our_mission_right_now_has_to_chang_uk_prime_minist_liz_truss_announc_uturn_on_corpor_tax_she_will_keep_the_plan_increas_in_order_to_reassur_the_market_of_our_fiscal_disciplin_httpstco2b6q3ihsun_httpstcoylgc8rjrvl', 0, '2022-10-14 13:44:40+00:00', 354, 51329496, 122), (3, 'uk_prime_minist_liz_truss_to_keep_increas_in_corpor_tax_in_revers_of_plan_in_minibudget_httpstcoe5ssbyfd3d', 0, '2022-10-14 13:40:48+00:00', 536, 51329496, 139), (4, 'uk_prime_minist_liz_truss_about

## next step in project
- need to conduct sentiment analysis on each tweet for each user in python
- from here we need to connect to an instance of PostgreSQL and store the sentiment analysis results
- Then we need to connect our database to powerBI and create a dashboard showing our results




In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#This is a simple plot that shows the top 20 words being used
#df.plot(20)

df = df[:20,]
plt.figure(figsize=(10,5))
# sns.barplot(df.index, df[0], alpha=0.8)
sns.barplot(x = df.values, y = df.index, alpha=0.8)
plt.title('Top Words Overall')
plt.ylabel('Word from Tweet', fontsize=12)
plt.xlabel('Count of Words', fontsize=12)
plt.show()



In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()



In [ ]:
def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text + ' - ' + ent.label_ + ' - ' + str(spacy.explain(ent.label_)))

In [ ]:
str1 = " " 
stem2 = str1.join(lines2)

stem2 = nlp(stem2)

label = [(X.text, X.label_) for X in stem2.ents]

df6 = pd.DataFrame(label, columns = ['Word','Entity'])

df7 = df6.where(df6['Entity'] == 'ORG')

df7 = df7['Word'].value_counts()


In [ ]:
df = df7[:20,]
plt.figure(figsize=(10,5))
sns.barplot(x = df.values, y = df.index, alpha=0.8)
plt.title('Top Organizations Mentioned')
plt.ylabel('Word from Tweet', fontsize=12)
plt.xlabel('Count of Words', fontsize=12)
plt.show()

In [ ]:
str1 = " " 
stem2 = str1.join(lines2)

stem2 = nlp(stem2)

label = [(X.text, X.label_) for X in stem2.ents]

df10 = pd.DataFrame(label, columns = ['Word','Entity'])

df10 = df10.where(df10['Entity'] == 'PERSON')

df11 = df10['Word'].value_counts()


In [ ]:
df = df11[:20,]
plt.figure(figsize=(10,5))
sns.barplot(x = df.values, y = df.index, alpha=0.8)
plt.title('Top People Mentioned')
plt.ylabel('Word from Tweet', fontsize=12)
plt.xlabel('Count of Words', fontsize=12)
plt.show()